In [7]:
df

,departureCity,arrivalCity,Distance,Demand
0,Amsterdam,London,222,1557
1,London,Frankfurt,397,1452
2,London,Amsterdam,222,1439
3,Paris,Amsterdam,267,1424
4,Frankfurt,Paris,297,1356
...,...,...,...,...
1635,Trondheim,Porto,1723,7
1636,Trondheim,Birmingham,879,7
1637,Bucharest,Trondheim,1452,7
1638,Trondheim,Lyon,1241,6


In [14]:
import gurobipy as gp
import numpy as np
import pandas as pd

df = pd.read_csv("C:/Users/mauri/Documents/Master/BAM/Management Science/Group assignment/flights.csv")
flights=[]
for index,row in df.iterrows():
    flights.append((row.departureCity,row.arrivalCity))
    flights.append((row.departureCity,row.arrivalCity))
flights=list(set(flights))#removing duplicates

aeroplaneSizes={'small': 50, 'medium': 100, 'large': 300}
aeroplaneCost={'small': 4.5, 'medium': 8, 'large': 20}
Fares = {}

Demand = {}
Distance = {}
for index,row in df.iterrows():
    Distance[row.departureCity,row.arrivalCity]=int(row.Distance)
    Demand[row.departureCity,row.arrivalCity]=int(row.Demand)

    
Planes={}
Tickets={}
SPlanes={}
MPlanes={}
LPlanes={}
Fare={}
for index,row in df.iterrows():
    l = 0
    m = 0
    s = 0
    t = 0
    nd = row.Demand

    while nd > 200: 
        if nd / aeroplaneSizes.get('large') > 0.66:
            l = l + 1
            if nd / aeroplaneSizes.get('large') > 1:
                t = t + 300
            else:
                t = t + nd
            nd = nd - 300
    Planes[row.departureCity,row.arrivalCity,"large"]= l
    while nd > 80:
        if nd / aeroplaneSizes.get('medium') > 0.79:
            m = m + 1

            if nd / aeroplaneSizes.get('medium') > 1:
                t = t + 100
            else:
                t = t + nd  
            nd = nd - 100
    Planes[row.departureCity,row.arrivalCity,"medium"] = m
    while nd > 44:
        if nd / aeroplaneSizes.get('small') > 0.89:
            s = s + 1

            if nd / aeroplaneSizes.get('small') > 1:
                t = t + 50
            else:
                t = t + nd  
            nd = nd - 50
    Planes[row.departureCity,row.arrivalCity,"small"]= s
    Planes[row.departureCity,row.arrivalCity,"tickets"]=t
    Tickets[row.departureCity,row.arrivalCity]=int(t)    
    SPlanes[row.departureCity,row.arrivalCity]= s
    MPlanes[row.departureCity,row.arrivalCity]= m
    LPlanes[row.departureCity,row.arrivalCity]= l

    
Profit={}
for t in Tickets:
    profit = ((
                    ((Tickets.get(t) * 0.1 * Distance.get(t)) 
                      - 
                      (SPlanes.get(t) * 4.5 * Distance.get(t) + MPlanes.get(t) * 8 * Distance.get(t) + MPlanes.get(t) * 20 * Distance.get(t)))
                   ))
    Profit[t]=profit

In [620]:
model=gp.Model()
x=model.addVars(Fare,ub=Demand,obj=Profit)
model.modelSense=gp.GRB.MAXIMIZE

#Cons?

print(Planes)
model.optimize()

print(sum(Profit.values()))

{('Paris', 'Madrid', 'large'): 2, ('Paris', 'Madrid', 'medium'): 1, ('Paris', 'Madrid', 'small'): 0, ('Paris', 'Madrid', 'tickets'): 700, ('Amsterdam', 'Madrid', 'large'): 2, ('Amsterdam', 'Madrid', 'medium'): 0, ('Amsterdam', 'Madrid', 'small'): 1, ('Amsterdam', 'Madrid', 'tickets'): 650, ('Paris', 'Istanbul', 'large'): 2, ('Paris', 'Istanbul', 'medium'): 0, ('Paris', 'Istanbul', 'small'): 1, ('Paris', 'Istanbul', 'tickets'): 650, ('Madrid', 'Paris', 'large'): 2, ('Madrid', 'Paris', 'medium'): 0, ('Madrid', 'Paris', 'small'): 1, ('Madrid', 'Paris', 'tickets'): 650, ('London', 'Manchester', 'large'): 2, ('London', 'Manchester', 'medium'): 0, ('London', 'Manchester', 'small'): 1, ('London', 'Manchester', 'tickets'): 650, ('Frankfurt', 'Madrid', 'large'): 2, ('Frankfurt', 'Madrid', 'medium'): 0, ('Frankfurt', 'Madrid', 'small'): 0, ('Frankfurt', 'Madrid', 'tickets'): 600, ('Moscow', 'Amsterdam', 'large'): 2, ('Moscow', 'Amsterdam', 'medium'): 0, ('Moscow', 'Amsterdam', 'small'): 0, ('Mos

In [15]:
Planes

{('Amsterdam', 'London', 'large'): 5,
 ('Amsterdam', 'London', 'medium'): 0,
 ('Amsterdam', 'London', 'small'): 1,
 ('Amsterdam', 'London', 'tickets'): 1550,
 ('London', 'Frankfurt', 'large'): 5,
 ('London', 'Frankfurt', 'medium'): 0,
 ('London', 'Frankfurt', 'small'): 0,
 ('London', 'Frankfurt', 'tickets'): 1452,
 ('London', 'Amsterdam', 'large'): 5,
 ('London', 'Amsterdam', 'medium'): 0,
 ('London', 'Amsterdam', 'small'): 0,
 ('London', 'Amsterdam', 'tickets'): 1439,
 ('Paris', 'Amsterdam', 'large'): 5,
 ('Paris', 'Amsterdam', 'medium'): 0,
 ('Paris', 'Amsterdam', 'small'): 0,
 ('Paris', 'Amsterdam', 'tickets'): 1424,
 ('Frankfurt', 'Paris', 'large'): 4,
 ('Frankfurt', 'Paris', 'medium'): 1,
 ('Frankfurt', 'Paris', 'small'): 1,
 ('Frankfurt', 'Paris', 'tickets'): 1350,
 ('Amsterdam', 'Frankfurt', 'large'): 4,
 ('Amsterdam', 'Frankfurt', 'medium'): 1,
 ('Amsterdam', 'Frankfurt', 'small'): 0,
 ('Amsterdam', 'Frankfurt', 'tickets'): 1293,
 ('Frankfurt', 'Amsterdam', 'large'): 4,
 ('Fran

In [5]:
print(sum(Profit.values()))

449278.5


In [20]:
for t in Tickets:
    print(((
                    ((Tickets.get(t) * 0.1 * Distance.get(t)) 
                      - 
                      (SPlanes.get(t) * 4.5 * Distance.get(t) + MPlanes.get(t) * 8 * Distance.get(t) + MPlanes.get(t) * 20 * Distance.get(t)))
                   )))

#(0.1 * 1150 * 213) - (20 * 4 * 213)

33411.0
57644.40000000001
31945.800000000003
38020.8
30442.5
22893.800000000003
27233.0
24750.600000000002
24495.0
14204.400000000001
28782.5
21384.0
33320.0
65352.0
38640.0
27468.0
55660.0
84942.0
39567.0
9845.2
53940.0
80280.0
17280.0
80280.0
53520.0
47040.0
84240.0
93360.0
53940.0
29160.0
93480.0
69195.6
91081.20000000001
22288.5
28207.200000000004
97355.5
21166.800000000003
39021.600000000006
39021.600000000006
17145.600000000002
64203.3
16146.0
21412.4
67482.40000000001
73616.0
66475.20000000001
25309.200000000004
80238.20000000001
80392.8
9563.4
-678.0
-1814.3999999999996
-5642.0
-3297.0
-2786.399999999998
-1455.2999999999993
-1996.7999999999993
-10870.399999999994
5225.0
6075.0
21312.5
3800.0
1412.5
2037.5
4975.0
7450.0
9612.5
11150.0
4012.5
4012.5
10075.0
4650.0
4275.0
5887.5
3737.5
5225.0
11820.0
1356.0
3684.0
1356.0
4572.0
10020.0
4296.0
3156.0
3072.0
7152.0
4512.0
3756.0
4512.0
3684.0
8616.0
7140.0
6996.0
8616.0
4644.0
7470.4000000000015
11327.5
9508.800000000003
12110.09999